<a href="https://colab.research.google.com/github/AkersJack/TextToLatex/blob/main/Load_Mamba_chat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install Libraries (ensure we have all the libraries we need)

In [ ]:
!pip install causal-conv1d
!pip install transformers accelerate bitsandbytes
!pip install mamba-ssm
!pip install datasets
!pip install trl
!pip install peft
!pip install numba
!pip install nltk
!pip install evaluate
!pip install rouge-score
!mkdir /content/data
!mkdir /content/mamba-chat
# BERT google-bert/bert-large-cased (needs chat training) or google-bert/bert-large-cased-whole-word-masking-finetuned-squad (trained)
# LLAMA meta-llama/Meta-Llama-3-8B or meta-llama/Llama-2-7b-chat-hf"
# GEMMA google/gemma-2b-it or IvanD2002/gemma-chatbot



  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 4.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cache

In [ ]:
from huggingface_hub import login
login(token = '')

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Import libraries and necessary classes. Also loads the Math dataset (both train and test splits)  Also prepairs for ROUGE and BLEU evaluation.


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, MambaForCausalLM, MambaConfig,TextStreamer, BitsAndBytesConfig
from mamba_ssm.models.mixer_seq_simple import MambaLMHeadModel

from datasets import load_dataset, Dataset

from trl import SFTTrainer, DataCollatorForCompletionOnlyLM
from peft import LoraConfig
import pandas as pd
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction # for BLEU score calculation
import evaluate # for ROUGE and perplexity score calculation



rouge = evaluate.load('rouge') #

perplexity = evaluate.load("perplexity", module_type="metric") # for perplexity evaluation


dataset = load_dataset("Kyudan/GTNT_PNT_WER")
#dataset_chat = load_dataset("HuggingFaceH4/ultrachat_200k")

#chatdataset = load_dataset("HuggingFaceH4/ultrachat_200k", split="train_sft")
#chatdataset_val = load_dataset("HuggingFaceH4/ultrachat_200k", split="test_sft")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/93618 [00:00<?, ? examples/s]

Combine the columns for the test dataset (sets up the test dataset)

In [ ]:
data = dataset['train']
data = data.remove_columns("WER of NT PNT")



# GT key
# NT and PNT are 2 different ways to explain the equation

train_1 = data['NT']
train_2 = data['PNT']
key = data["GT"]

datad = {}

for x in range(0, len(train_1)):
  datad[train_1[x]] = key[x]
  datad[train_2[x]] = key[x]


input_list = list(datad.keys())

output_list = []
for x in input_list:
  output_list.append(datad[x])





dataf = pd.DataFrame({"Input":input_list, "Output":output_list})

dataf

#datasetm = Dataset.from_pandas(dataf)

test_data = dataf[149479:] # Seperate out the last 20% of the data


In [ ]:
# len(dataf)
# 149478 train (80%)
# 37370 Test (20%)

#test_data = dataf[149479:] # Seperate out the last 20% of the data
test_data[300000:]
#test_data["Input"].iloc[0] # Get the first item of the input column
#test_data["Output"].iloc[0] # Get the first item of the output column


,Input,Output


Sets up the math dataset for training (shouldn't need to run this when testing/evaluating)

In [ ]:
roleDict_base = {
    "content":"", # Content here
    "role":"", # Role here (assistant or user)
}

textlist = []
data = dataset['train']

data = data.remove_columns("WER of NT PNT")

train_1 = data['NT']
train_2 = data['PNT']
key = data["GT"]
for x in range(0, len(train_1)):
  eq_list = []
  roleDict = roleDict_base.copy()
  roleDict["content"] = train_1[x]
  roleDict["role"] = "user"
  eq_list.append(roleDict)
  roleDict = roleDict_base.copy()
  roleDict["content"] = key[x]
  roleDict["role"] = "assistant"
  eq_list.append(roleDict)
  textlist.append(eq_list)
  eq_list = []
  roleDict = roleDict_base.copy()
  roleDict["content"] = train_2[x]
  roleDict["role"] = "user"
  eq_list.append(roleDict)
  roleDict = roleDict_base.copy()
  roleDict["content"] = key[x]
  roleDict["role"] = "assistant"
  eq_list.append(roleDict)
  textlist.append(eq_list)


messages_dataset = {"messages": textlist}
dataset_test = Dataset.from_dict(messages_dataset)

**GEMMA**



In [ ]:
device = "cuda"
model_name = "IvanD2002/gemma-chatbot"
tokenizer = AutoTokenizer.from_pretrained(model_name)
#tokenizer.eos_token = "<|endoftext|>"
#tokenizer.pad_token = tokenizer.eos_token
#tokenizer.chat_template = AutoTokenizer.from_pretrained("HuggingFaceH4/zephyr-7b-beta").chat_template
model = AutoModelForCausalLM.from_pretrained(model_name,torch_dtype=torch.bfloat16)
model.to(device)

**BERT**

In [ ]:
device = "cuda"
model_name = "google-bert/bert-large-cased-whole-word-masking-finetuned-squad"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.eos_token = "<|endoftext|>"
tokenizer.pad_token = tokenizer.eos_token
tokenizer.chat_template = AutoTokenizer.from_pretrained("HuggingFaceH4/zephyr-7b-beta").chat_template
model = AutoModelForCausalLM.from_pretrained(model_name) # This for the custom model
model.to(device)


NameError: name 'AutoTokenizer' is not defined

**Mamba Chat Model**

In [ ]:
device = "cuda"

#model_name = "state-spaces/mamba-370m-hf"
# tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/NLP_Project/mamba-chat-math2/checkpoint-8000") # this for custom model tokenizer
#tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf", use_fast=True, use_auth_token=True) # This for Llama model tokenizer

tokenizer = AutoTokenizer.from_pretrained("havenhq/mamba-chat")

#tokenizer = AutoTokenizer.from_pretrained(model_name)



#quantization_config = BitsAndBytesConfig(load_in_4bit=True) # https://discuss.huggingface.co/t/correct-usage-of-bitsandbytesconfig/33809
#quantization_config = BitsAndBytesConfig() # https://discuss.huggingface.co/t/correct-usage-of-bitsandbytesconfig/33809

#model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", quantization_config=quantization_config)
#tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True, use_auth_token=True)

tokenizer.eos_token = "<|endoftext|>"
tokenizer.pad_token = tokenizer.eos_token
tokenizer.chat_template = AutoTokenizer.from_pretrained("HuggingFaceH4/zephyr-7b-beta").chat_template

# model = AutoModelForCausalLM.from_pretrained("/content/drive/MyDrive/NLP_Project/mamba-chat-math2/checkpoint-8000", residual_in_fp32 = False) # This for the custom model
#model = AutoModelForCausalLM.from_pretrained(model_name) # This for the custom model

#quantization_config = BitsAndBytesConfig(load_in_4bit=True) # https://discuss.huggingface.co/t/correct-usage-of-bitsandbytesconfig/33809

#model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf", quantization_config=quantization_config) # This for the Llama model




model = MambaLMHeadModel.from_pretrained("havenhq/mamba-chat", device="cuda", dtype=torch.float16)
model.to(device)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/4.79k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/131 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


...

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/5.55G [00:00<?, ?B/s]

MambaLMHeadModel(
  (backbone): MixerModel(
    (embedding): Embedding(50280, 2560)
    (layers): ModuleList(
      (0-63): 64 x Block(
        (mixer): Mamba(
          (in_proj): Linear(in_features=2560, out_features=10240, bias=False)
          (conv1d): Conv1d(5120, 5120, kernel_size=(4,), stride=(1,), padding=(3,), groups=5120)
          (act): SiLU()
          (x_proj): Linear(in_features=5120, out_features=192, bias=False)
          (dt_proj): Linear(in_features=160, out_features=5120, bias=True)
          (out_proj): Linear(in_features=5120, out_features=2560, bias=False)
        )
        (norm): RMSNorm()
      )
    )
    (norm_f): RMSNorm()
  )
  (lm_head): Linear(in_features=2560, out_features=50280, bias=False)
)

**Llama 2** Loads the model, tokenizer, and sets up the chat_template (along with setting padding, and eos_token)



In [ ]:




device = "cuda"

model_name = "meta-llama/Llama-2-7b-chat-hf"
# tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/NLP_Project/mamba-chat-math2/checkpoint-8000") # this for custom model tokenizer
#tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf", use_fast=True, use_auth_token=True) # This for Llama model tokenizer

#tokenizer = AutoTokenizer.from_pretrained("havenhq/mamba-chat")


#quantization_config = BitsAndBytesConfig(load_in_4bit=True) # https://discuss.huggingface.co/t/correct-usage-of-bitsandbytesconfig/33809
#quantization_config = BitsAndBytesConfig() # https://discuss.huggingface.co/t/correct-usage-of-bitsandbytesconfig/33809

#model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", quantization_config=quantization_config)
#tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True, use_auth_token=True)

# tokenizer.eos_token = "<|endoftext|>"
# tokenizer.pad_token = tokenizer.eos_token
# tokenizer.chat_template = AutoTokenizer.from_pretrained("HuggingFaceH4/zephyr-7b-beta").chat_template

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name,torch_dtype=torch.bfloat16)
model.to(device)

# model = AutoModelForCausalLM.from_pretrained("/content/drive/MyDrive/NLP_Project/mamba-chat-math2/checkpoint-8000", residual_in_fp32 = False) # This for the custom model
#quantization_config = BitsAndBytesConfig(load_in_4bit=True) # https://discuss.huggingface.co/t/correct-usage-of-bitsandbytesconfig/33809

#model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf", quantization_config=quantization_config) # This for the Llama model




# model = MambaLMHeadModel.from_pretrained("havenhq/mamba-chat", device="cuda", dtype=torch.float16)
#model.to(device)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_head):

**Custom Mamba Model** Loads the model, tokenizer, and sets up the chat_template (along with setting padding, and eos_token)


In [ ]:
device = "cuda"

#model_name = "state-spaces/mamba-370m-hf"
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/NLP_Project/mamba-chat-math2/checkpoint-8000") # this for custom model tokenizer
#tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf", use_fast=True, use_auth_token=True) # This for Llama model tokenizer

#tokenizer = AutoTokenizer.from_pretrained("havenhq/mamba-chat")

#quantization_config = BitsAndBytesConfig() # https://discuss.huggingface.co/t/correct-usage-of-bitsandbytesconfig/33809
#model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", quantization_config=quantization_config)
#tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True, use_auth_token=True)

tokenizer.eos_token = "<|endoftext|>"
tokenizer.pad_token = tokenizer.eos_token
tokenizer.chat_template = AutoTokenizer.from_pretrained("HuggingFaceH4/zephyr-7b-beta").chat_template

model = AutoModelForCausalLM.from_pretrained("/content/drive/MyDrive/NLP_Project/mamba-chat-math2/checkpoint-8000", residual_in_fp32 = False) # This for the custom model
#quantization_config = BitsAndBytesConfig(load_in_4bit=True) # https://discuss.huggingface.co/t/correct-usage-of-bitsandbytesconfig/33809

#model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf", quantization_config=quantization_config) # This for the Llama model




# model = MambaLMHeadModel.from_pretrained("havenhq/mamba-chat", device="cuda", dtype=torch.float16)
model.to(device)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


MambaForCausalLM(
  (backbone): MambaModel(
    (embeddings): Embedding(50280, 1536)
    (layers): ModuleList(
      (0-47): 48 x MambaBlock(
        (norm): MambaRMSNorm()
        (mixer): MambaMixer(
          (conv1d): Conv1d(3072, 3072, kernel_size=(4,), stride=(1,), padding=(3,), groups=3072)
          (act): SiLU()
          (in_proj): Linear(in_features=1536, out_features=6144, bias=False)
          (x_proj): Linear(in_features=3072, out_features=128, bias=False)
          (dt_proj): Linear(in_features=96, out_features=3072, bias=True)
          (out_proj): Linear(in_features=3072, out_features=1536, bias=False)
        )
      )
    )
    (norm_f): MambaRMSNorm()
  )
  (lm_head): Linear(in_features=1536, out_features=50280, bias=False)
)

Run the Model (Custom Mamba Model Only)

In [ ]:
messages = []
test_count = 0
rouge = evaluate.load('rouge')
bleu = 0
rouge1 = 0 # {'rouge1': 1.0, 'rouge2': 1.0, 'rougeL': 1.0, 'rougeLsum': 1.0}
rouge2 = 0
rougeL = 0
rougeLsum = 0
predictions = []
ref_list = []

while True:
    user_message = test_data["Input"].iloc[test_count] # Get the first item of the input column (for the Mamba model)



    ref = test_data["Output"].iloc[test_count] # Get the first item of the output column
    ref_list.append(ref)

    #user_message = input("\nYour message: ")

    messages.append(dict(
        role="user",
        content=user_message
    ))

    #streamer = TextStreamer(tokenizer) # Don't need this it just streams the text (not fully implemented)
    input_ids = tokenizer.apply_chat_template(messages, return_tensors="pt", add_generation_prompt=True).to("cuda")

    #out = model.generate(input_ids=input_ids, max_length=2000, temperature=0.9, top_p=0.7, eos_token_id=tokenizer.eos_token_id)
    #out = model.generate(input_ids=input_ids, temperature=0.9, top_p=0.7, eos_token_id=tokenizer.eos_token_id, streamer=streamer)
    out = model.generate(input_ids=input_ids, max_length=2000, temperature=0.9, top_p=0.7, eos_token_id=tokenizer.eos_token_id, do_sample=True) # Mamba
    #out = model.generate(input_ids=input_ids, temperature=0.9, top_p=0.7, eos_token_id=tokenizer.eos_token_id) # Llama




    decoded = tokenizer.batch_decode(out)
    messages.append(dict(
        role="assistant",
        content=decoded[0].split("<|assistant|>\n")[-1])
    )
    output = decoded[0].split("<|assistant|>\n")[-1]
    messages = []

    # Evaluate ROUGE scores
    prediction = output.split("<|endoftext|>")[0]
    predictions.append(prediction)
    reference = ref


    print(f"Time: {test_count}/{100}")
    print("Model:", prediction)
    #print(f"ROUGE: {rouge.compute(predictions=[prediction], references=[reference])}")
    rouge_dict = rouge.compute(predictions=[prediction], references=[reference])
    rouge1 += rouge_dict['rouge1']
    rouge2 += rouge_dict['rouge2']
    rougeL += rouge_dict['rougeL']
    rougeLsum += rouge_dict['rougeLsum']

    #print('BLEU score -> {}'.format(sentence_bleu([ref.split()], prediction.split())))
    bleu += sentence_bleu([reference.split()], prediction.split())
    print(f"BLEU: {bleu/(test_count + 1)}")
    print(f"rogue1: {rouge1/(test_count + 1)}")
    print(f"rogue2: {rouge2/(test_count + 1)}")
    print(f"rogueL: {rougeL/(test_count + 1)}")
    print(f"rougeLsum: {rougeLsum/(test_count + 1)}")



    # print(f"BLEU Split: {prediction.split()}")
    # print(f"Reference Split: {ref.split()}")
    #if test_count == 37365: # Automatically exit
    if test_count == 100:
      break
    test_count += 1






print("Done")

rouge_dict = rouge.compute(predictions=predictions, references=ref_list)
rouge1 = rouge_dict['rouge1']
rouge2 = rouge_dict['rouge2']
rougeL = rouge_dict['rougeL']
rougeLsum = rouge_dict['rougeLsum']

perp = perplexity.compute(predictions=predictions, references=ref_list, model_id="state-spaces/mamba-790m-hf")
print(f"Perplexity: {perp}")

print(f"rogue1: {rouge1}")
print(f"rogue2: {rouge2}")
print(f"rogueL: {rougeL}")
print(f"rougeLsum: {rougeLsum}")





Time: 0/250
Model: When it comes to $\epsilon = \frac{a^2 + 2^2}{c^2 + 4^2}$we can infer that
BLEU: 1.0
rogue1: 1.0
rogue2: 1.0
rogueL: 1.0
rougeLsum: 1.0
Time: 1/250
Model: When it comes to $\epsilon = \frac{a^2 + 2^2}{c^2 + 4^2}$we can infer that
BLEU: 1.0
rogue1: 1.0
rogue2: 1.0
rogueL: 1.0
rougeLsum: 1.0
Time: 2/250
Model: Consider the equation $R = \frac{W}{R - T - T - T - T}$is fundamental in study,
BLEU: 0.8190694297312137
rogue1: 0.9607843137254902
rogue2: 0.875
rogueL: 0.9607843137254902
rougeLsum: 0.9607843137254902
Time: 3/250
Model: Consider the equation $R = \frac{W}{R - T - T - T - T}$is fundamental in study,
BLEU: 0.7286041445968205
rogue1: 0.9411764705882353
rogue2: 0.8125
rogueL: 0.9411764705882353
rougeLsum: 0.9411764705882353
Time: 4/250
Model: Given the formula $Q = V * -17$known as the ideal law
BLEU: 0.7096926412183937
rogue1: 0.9442455242966752
rogue2: 0.8214285714285715
rogueL: 0.9442455242966752
rougeLsum: 0.9442455242966752
Time: 5/250
Model: Given the formula

/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


BLEU: 0.5975014048280934
rogue1: 0.9390388212390731
rogue2: 0.8190737833594978
rogueL: 0.9390388212390731
rougeLsum: 0.9390388212390731
Time: 7/250
Model: Given the formula $H_{ s l } = ( D_{ X_{ 1 } } - \frac { 1 } { 2 } B_{ 0 } X_{ 2 } )^{ 2 } + ( D_{ X_{ 2 } } + \frac { 1 } { 2 } B_{ 0 } X_{ 1 } )^{ 2 },$plays a crucial role.
BLEU: 0.5228137292245818
rogue1: 0.9335992670916516
rogue2: 0.8128434065934067
rogueL: 0.9335992670916516
rougeLsum: 0.9335992670916516
Time: 8/250
Model: When it comes to $\int \sec^2 \phi \, d\phi = \tan \phi + x$known as the ideal law
BLEU: 0.5388653780656956
rogue1: 0.9409771263036902
rogue2: 0.8336385836385838
rogueL: 0.9409771263036902
rougeLsum: 0.9409771263036902
Time: 9/250
Model: When it comes to $\int \sec^2 \phi \, d\phi = \tan \phi + x$known as the ideal law
BLEU: 0.5517066971385866
rogue1: 0.9468794136733212
rogue2: 0.8502747252747254
rogueL: 0.9468794136733212
rougeLsum: 0.9468794136733212
Time: 10/250
Model: The equation $y = mc \cdot x$known as

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


  0%|          | 0/16 [00:00<?, ?it/s]

Perplexity: {'perplexities': [18.683866500854492, 18.683866500854492, 53.546424865722656, 53.546424865722656, 345.9563293457031, 345.9563293457031, 8.630608558654785, 7.279150485992432, 44.3663330078125, 44.3663330078125, 142.40843200683594, 245.66732788085938, 15.120253562927246, 16.83070182800293, 97.81658172607422, 69.3617172241211, 8.836302757263184, 9.314143180847168, 28.490190505981445, 28.849061965942383, 29.582006454467773, 29.582006454467773, 151.47520446777344, 151.47520446777344, 143.394287109375, 143.394287109375, 30.337020874023438, 30.337020874023438, 50.23957824707031, 50.59873580932617, 64.22356414794922, 36.76139831542969, 99.58269500732422, 180.5665283203125, 52.18124008178711, 51.779537200927734, 111.14688110351562, 111.14688110351562, 17.150909423828125, 20.185564041137695, 9.858115196228027, 10.79526424407959, 13.770112037658691, 22.868539810180664, 52.550350189208984, 58.691856384277344, 37.75117111206055, 48.87905502319336, 49.814205169677734, 49.814205169677734,

Mamba Chat

In [ ]:
messages = []
test_count = 0
rouge = evaluate.load('rouge')
bleu = 0
rouge1 = 0 # {'rouge1': 1.0, 'rouge2': 1.0, 'rougeL': 1.0, 'rougeLsum': 1.0}
rouge2 = 0
rougeL = 0
rougeLsum = 0
predictions = []
ref_list = []
while True:
    #user_message = test_data["Input"].iloc[test_count] # Get the first item of the input column (for the Mamba model)

    # Only use this if you are not using the custom Mamba model
    #base_message = "You are a helpful assistant who converts text based math equations into LaTeX. When given a prompt you are going to convert that prompt into a LaTeX wquation with the following form: Answer: Latex equation here. So if you are given the prompt: Convert 1 + 1 into LaTeX your answer should be: Answer: $1 + 1$. You will not output any other information only the LaTeX equation in the format shown."
    #example_2 = "Another example is: 2 divided by 2. This should be converted to: Answer: $\frac{2}{2}$. Another Example is: 2 - 2. This should be converted to: Answer: $2 - 2$"
    #base_message = "Convert the following into LaTeX with the output being of the form: Answer: LaTeX equation here. Do not add any information, explanation, or reasoning before or after the equation. Only output the LaTeX equation in the format specified."
    # messages.append(dict(
    #     role="user",
    #     content=base_message
    # ))
    # messages.append(dict(
    #     role="user",
    #     content=example_2
    # ))

    #user_message = test_data["Input"].iloc[test_count]
    user_message = "Convert the following into LaTeX. Don't output any additional information before or after the converted text prompt. Only convert the given prompt into LaTeX: " + test_data["Input"].iloc[test_count]

    ref = test_data["Output"].iloc[test_count] # Get the first item of the output column
    ref_list.append(ref)

    #user_message = input("\nYour message: ")



    messages.append(dict(
        role="user",
        content=user_message
    ))

    #streamer = TextStreamer(tokenizer) # Don't need this it just streams the text (not fully implemented)
    #input_ids = tokenizer.apply_chat_template(messages, return_tensors="pt", add_generation_prompt=True).to("cuda")
    model_inputs = tokenizer.apply_chat_template(messages, return_tensors="pt", add_generation_prompt=True).to("cuda") # Mambas
    #input_ids = tokenizer(user_message, return_tensors="pt").to("cuda")

    #prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)


    #inputs = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")


    #output = model.generate(input_ids=model_inputs, max_length=2000, temperature=0.9, top_p=0.7, eos_token_id=tokenizer.eos_token_id, do_sample=True) # Mamba Base (might remove do_sample=True)
    #out = model.generate(input_ids=input_ids, temperature=0.9, top_p=0.7, eos_token_id=tokenizer.eos_token_id, streamer=streamer)
    output = model.generate(input_ids=model_inputs, max_length=2000, temperature=0.9, top_p=0.7, eos_token_id=tokenizer.eos_token_id) # Mamba chat)
    #out = model.generate(input_ids=input_ids, temperature=0.9, top_p=0.7, eos_token_id=tokenizer.eos_token_id) # Llama
    #output = model.generate(**input_ids, max_length=2000)
    #output = model.generate(input_ids=inputs.to("cuda"), max_new_tokens=150)# GEMMA chat



    #model_inputs = tokenizer(user_message, return_tensors="pt").to("cuda:0")


    #output = model.generate(model_inputs, max_length=2000, eos_token_id=tokenizer.eos_token_id)

    #print(tokenizer.decode(output[0], skip_special_tokens=True))

    prediction = tokenizer.decode(output[0])


    decoded = tokenizer.batch_decode(output)# Mambas
    # messages.append(dict(
    #     role="assistant",
    #     content=decoded[0].split("<|assistant|>\n")[-1])
    # )
    #output = decoded[0].split("<|assistant|>\n")[-1]
    messages = []

    # # Evaluate ROUGE scores
    #prediction = output.split("<|endoftext|>")[0]


    reference = ref



    #print("Model:", output.split("<|endoftext|>")[0])
    #base_pred = output.split("<|endoftext|>")[0]
    # prediction = prediction.split("<bos>")[0]
    #prediction = prediction.split("<eos>")[0]
    #prediction = prediction.split("<start_of_turn>model")[1]

    messages.append(dict(
        role="assistant",
        content=decoded[0].split("<|assistant|>\n")[-1])
    )
    output = decoded[0].split("<|assistant|>\n")[-1]

    # Evaluate ROUGE scores
    prediction = output.split("<|endoftext|>")[0]
    predictions.append(prediction)




    print(f"Time: {test_count}/{10}")
    print("Model:", prediction)
    #print(f"ROUGE: {rouge.compute(predictions=[prediction], references=[reference])}")




    rouge_dict = rouge.compute(predictions=[prediction], references=[reference])
    rouge1 += rouge_dict['rouge1']
    rouge2 += rouge_dict['rouge2']
    rougeL += rouge_dict['rougeL']
    rougeLsum += rouge_dict['rougeLsum']

    #print('BLEU score -> {}'.format(sentence_bleu([ref.split()], prediction.split())))
    bleu += sentence_bleu([reference.split()], prediction.split())

    print(f"BLEU: {bleu/(test_count + 1)}")
    print(f"rogue1: {rouge1/(test_count + 1)}")
    print(f"rogue2: {rouge2/(test_count + 1)}")
    print(f"rogueL: {rougeL/(test_count + 1)}")
    print(f"rougeLsum: {rougeLsum/(test_count + 1)}")


    #print(base_pred.split("</s>")[0])

    # print(rouge.compute(predictions=[prediction], references=[reference]))
    # #if test_count == 37365: # Automatically exit
    if test_count == 10:
      break
    test_count += 1





print("Done")

rouge_dict = rouge.compute(predictions=predictions, references=ref_list)
rouge1 = rouge_dict['rouge1']
rouge2 = rouge_dict['rouge2']
rougeL = rouge_dict['rougeL']
rougeLsum = rouge_dict['rougeLsum']

perp = perplexity.compute(predictions=predictions, references=ref_list, model_id="state-spaces/mamba-2.8b-hf")
print(f"Perplexity: {perp}")

print(f"rogue1: {rouge1}")
print(f"rogue2: {rouge2}")
print(f"rogueL: {rougeL}")
print(f"rougeLsum: {rougeLsum}")





Time: 0/10
Model: $\epsilon = \frac{a^2 + 2^2}{c^2 + 4^2}$
BLEU: 0.29765372490051634
rogue1: 0.7142857142857143
rogue2: 0.6923076923076924
rogueL: 0.7142857142857143
rougeLsum: 0.7142857142857143
Time: 1/10
Model: $\epsilon = \frac{a^2 + 2^2}{c^2 + 4^2}$
BLEU: 0.29765372490051634
rogue1: 0.7142857142857143
rogue2: 0.6923076923076924
rogueL: 0.7142857142857143
rougeLsum: 0.7142857142857143
Time: 2/10
Model: $\epsilon = \frac{a^2 + 2^2}{c^2 + 4^2}$

$r = \frac{w}{r - t - t - t}$

$r = \frac{w}{r - t - t - t}$

$r = \frac{w}{r - t - t - t}$

$r = \frac{w}{r - t - t - t}$

$r = \frac{w}{r - t - t - t}$

$r = \frac{w}{r - t - t - t}$

$r = \frac{w}{r - t - t - t}$

$r = \frac{w}{r - t - t - t}$

$r = \frac{w}{r - t - t - t}$

$r = \frac{w}{r - t - t - t}$

$r = \frac{w}{r - t - t - t}$

$r = \frac{w}{r - t - t - t}$

$r = \frac{w}{r - t - t - t}$

$r = \frac{w}{r - t - t - t}$

$r = \frac{w}{r - t - t - t}$

$r = \frac{w}{r - t - t - t}$

$r = \frac{w}{r - t - t - t}$

$r = \frac{w}{r - t -

/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

Time: 3/10
Model: $\epsilon = \frac{a^2 + 2^2}{c^2 + 4^2
BLEU: 0.14882686245025817
rogue1: 0.3796473943479388
rogue2: 0.3481327379744267
rogueL: 0.3796473943479388
rougeLsum: 0.3789894996110967
Time: 4/10
Model: $\epsilon = \frac{a^2 + 2^2}{c^2 + 4^2}$

$q = v \times -17$

$q = \frac{v}{c^2 + 4^2} \times -17$

$q = \frac{v}{c^2 + 4^2} \times -17$

$q = \frac{v}{c^2 + 4^2} \times -17$

$q = \frac{v}{c^2 + 4^2} \times -17$

$q = \frac{v}{c^2 + 4^2} \times -17$

$q = \frac{v}{c^2 + 4^2} \times -17$

$q = \frac{v}{c^2 + 4^2} \times -17$

$q = \frac{v}{c^2 + 4^2} \times -17$

$q = \frac{v}{c^2 + 4^2} \times -17$

$q = \frac{v}{c^2 + 4^2} \times -17$

$q = \frac{v}{c^2 + 4^2} \times -17$

$q = \frac{v}{c^2 + 4^2} \times -17$

$q = \frac{v}{c^2 + 4^2} \times -17$

$q = \frac{v}{c^2 + 4^2} \times -17$

$q = \frac{v}{c^2 + 4^2} \times -17$

$q = \frac{v}{c^2 + 4^2} \times -17$

$q = \frac{v}{c^2 + 4^2} \times -17$

$q = \frac{v}{c^2 + 4^2} \times -17$

$q = \frac{v}{c^2 + 4^2} \times -17$

$q =

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/4.79k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


  0%|          | 0/1 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 3.95 GiB. GPU 0 has a total capacity of 22.17 GiB of which 2.37 GiB is free. Process 8122 has 19.79 GiB memory in use. Of the allocated memory 19.50 GiB is allocated by PyTorch, and 59.20 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

GEMMA

In [ ]:
messages = []
test_count = 0
rouge = evaluate.load('rouge')
bleu = 0
rouge1 = 0 # {'rouge1': 1.0, 'rouge2': 1.0, 'rougeL': 1.0, 'rougeLsum': 1.0}
rouge2 = 0
rougeL = 0
rougeLsum = 0
predictions = []
ref_list = []

while True:
    #user_message = test_data["Input"].iloc[test_count] # Get the first item of the input column (for the Mamba model)

    # Only use this if you are not using the custom Mamba model
    #base_message = "You are a helpful assistant who converts text based math equations into LaTeX. When given a prompt you are going to convert that prompt into a LaTeX wquation with the following form: Answer: Latex equation here. So if you are given the prompt: Convert 1 + 1 into LaTeX your answer should be: Answer: $1 + 1$. You will not output any other information only the LaTeX equation in the format shown."
    #example_2 = "Another example is: 2 divided by 2. This should be converted to: Answer: $\frac{2}{2}$. Another Example is: 2 - 2. This should be converted to: Answer: $2 - 2$"
    #base_message = "Convert the following into LaTeX with the output being of the form: Answer: LaTeX equation here. Do not add any information, explanation, or reasoning before or after the equation. Only output the LaTeX equation in the format specified."
    # messages.append(dict(
    #     role="user",
    #     content=base_message
    # ))
    # messages.append(dict(
    #     role="user",
    #     content=example_2
    # ))

    #user_message = test_data["Input"].iloc[test_count]
    user_message = "Convert the following into LaTeX. Don't output any additional information before or after the converted text prompt. Only convert the given prompt into LaTeX: " + test_data["Input"].iloc[test_count]

    ref = test_data["Output"].iloc[test_count] # Get the first item of the output column
    ref_list.append(ref)

    #user_message = input("\nYour message: ")



    messages.append(dict(
        role="user",
        content=user_message
    ))

    #streamer = TextStreamer(tokenizer) # Don't need this it just streams the text (not fully implemented)
    #input_ids = tokenizer.apply_chat_template(messages, return_tensors="pt", add_generation_prompt=True).to("cuda")
    #model_inputs = tokenizer.apply_chat_template(messages, return_tensors="pt", add_generation_prompt=True).to("cuda") # Mambas
    #input_ids = tokenizer(user_message, return_tensors="pt").to("cuda")

    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)


    inputs = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")


    #output = model.generate(input_ids=model_inputs, max_length=2000, temperature=0.9, top_p=0.7, eos_token_id=tokenizer.eos_token_id, do_sample=True) # Mamba Base (might remove do_sample=True)
    #out = model.generate(input_ids=input_ids, temperature=0.9, top_p=0.7, eos_token_id=tokenizer.eos_token_id, streamer=streamer)
    #output = model.generate(input_ids=model_inputs, max_length=2000, temperature=0.9, top_p=0.7, eos_token_id=tokenizer.eos_token_id) # Mamba chat)
    #out = model.generate(input_ids=input_ids, temperature=0.9, top_p=0.7, eos_token_id=tokenizer.eos_token_id) # Llama
    #output = model.generate(**input_ids, max_length=2000)
    output = model.generate(input_ids=inputs.to("cuda"), max_new_tokens=150)# GEMMA chat



    #model_inputs = tokenizer(user_message, return_tensors="pt").to("cuda:0")


    #output = model.generate(model_inputs, max_length=2000, eos_token_id=tokenizer.eos_token_id)

    #print(tokenizer.decode(output[0], skip_special_tokens=True))

    prediction = tokenizer.decode(output[0])


    # decoded = tokenizer.batch_decode(output)# Mambas
    # messages.append(dict(
    #     role="assistant",
    #     content=decoded[0].split("<|assistant|>\n")[-1])
    # )
    #output = decoded[0].split("<|assistant|>\n")[-1]
    messages = []

    # # Evaluate ROUGE scores
    #prediction = output.split("<|endoftext|>")[0]


    reference = ref


    #print("Model:", output.split("<|endoftext|>")[0])
    #base_pred = output.split("<|endoftext|>")[0]
    # prediction = prediction.split("<bos>")[0]
    prediction = prediction.split("<eos>")[0]
    prediction = prediction.split("<start_of_turn>model")[1]
    predictions.append(prediction)


    print(f"Time: {test_count}/{100}")
    print("Model:", prediction)
    #print(f"ROUGE: {rouge.compute(predictions=[prediction], references=[reference])}")




    rouge_dict = rouge.compute(predictions=[prediction], references=[reference])
    rouge1 += rouge_dict['rouge1']
    rouge2 += rouge_dict['rouge2']
    rougeL += rouge_dict['rougeL']
    rougeLsum += rouge_dict['rougeLsum']

    #print('BLEU score -> {}'.format(sentence_bleu([ref.split()], prediction.split())))
    bleu += sentence_bleu([reference.split()], prediction.split())

    print(f"BLEU: {bleu/(test_count + 1)}")
    print(f"rogue1: {rouge1/(test_count + 1)}")
    print(f"rogue2: {rouge2/(test_count + 1)}")
    print(f"rogueL: {rougeL/(test_count + 1)}")
    print(f"rougeLsum: {rougeLsum/(test_count + 1)}")


    #print(base_pred.split("</s>")[0])

    # print(rouge.compute(predictions=[prediction], references=[reference]))
    # #if test_count == 37365: # Automatically exit
    if test_count == 100:
      break
    test_count += 1

print("Done")

rouge_dict = rouge.compute(predictions=predictions, references=ref_list)
rouge1 = rouge_dict['rouge1']
rouge2 = rouge_dict['rouge2']
rougeL = rouge_dict['rougeL']
rougeLsum = rouge_dict['rougeLsum']

perp = perplexity.compute(predictions=predictions, references=ref_list, model_id="IvanD2002/gemma-chatbot")
print(f"Perplexity: {perp}")

print(f"rogue1: {rouge1}")
print(f"rogue2: {rouge2}")
print(f"rogueL: {rougeL}")
print(f"rougeLsum: {rougeLsum}")


LLAMA

In [ ]:
messages = []
test_count = 0
rouge = evaluate.load('rouge')
bleu = 0
rouge1 = 0 # {'rouge1': 1.0, 'rouge2': 1.0, 'rougeL': 1.0, 'rougeLsum': 1.0}
rouge2 = 0
rougeL = 0
rougeLsum = 0
predictions = []
ref_list = []

while True:
    #user_message = test_data["Input"].iloc[test_count] # Get the first item of the input column (for the Mamba model)

    # Only use this if you are not using the custom Mamba model
    #base_message = "You are a helpful assistant who converts text based math equations into LaTeX. When given a prompt you are going to convert that prompt into a LaTeX wquation with the following form: Answer: Latex equation here. So if you are given the prompt: Convert 1 + 1 into LaTeX your answer should be: Answer: $1 + 1$. You will not output any other information only the LaTeX equation in the format shown."
    #example_2 = "Another example is: 2 divided by 2. This should be converted to: Answer: $\frac{2}{2}$. Another Example is: 2 - 2. This should be converted to: Answer: $2 - 2$"
    #base_message = "Convert the following into LaTeX with the output being of the form: Answer: LaTeX equation here. Do not add any information, explanation, or reasoning before or after the equation. Only output the LaTeX equation in the format specified."
    # messages.append(dict(
    #     role="user",
    #     content=base_message
    # ))
    # messages.append(dict(
    #     role="user",
    #     content=example_2
    # ))

    #user_message = test_data["Input"].iloc[test_count]
    user_message = "Convert the following into LaTeX. Don't output any additional information before or after the converted text prompt. Only convert the given prompt into LaTeX: " + test_data["Input"].iloc[test_count]

    ref = test_data["Output"].iloc[test_count] # Get the first item of the output column
    ref_list.append(ref)

    #user_message = input("\nYour message: ")



    messages.append(dict(
        role="user",
        content=user_message
    ))

    #streamer = TextStreamer(tokenizer) # Don't need this it just streams the text (not fully implemented)
    #input_ids = tokenizer.apply_chat_template(messages, return_tensors="pt", add_generation_prompt=True).to("cuda")
    #model_inputs = tokenizer.apply_chat_template(messages, return_tensors="pt", add_generation_prompt=True).to("cuda") # Mambas
    #input_ids = tokenizer(user_message, return_tensors="pt").to("cuda")

    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)


    inputs = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")


    #output = model.generate(input_ids=model_inputs, max_length=2000, temperature=0.9, top_p=0.7, eos_token_id=tokenizer.eos_token_id, do_sample=True) # Mamba Base (might remove do_sample=True)
    #out = model.generate(input_ids=input_ids, temperature=0.9, top_p=0.7, eos_token_id=tokenizer.eos_token_id, streamer=streamer)
    #output = model.generate(input_ids=model_inputs, max_length=2000, temperature=0.9, top_p=0.7, eos_token_id=tokenizer.eos_token_id) # Mamba chat)
    #out = model.generate(input_ids=input_ids, temperature=0.9, top_p=0.7, eos_token_id=tokenizer.eos_token_id) # Llama
    #output = model.generate(**input_ids, max_length=2000)
    output = model.generate(input_ids=inputs.to("cuda"), max_new_tokens=150)# GEMMA chat



    #model_inputs = tokenizer(user_message, return_tensors="pt").to("cuda:0")


    #output = model.generate(model_inputs, max_length=2000, eos_token_id=tokenizer.eos_token_id)

    #print(tokenizer.decode(output[0], skip_special_tokens=True))

    prediction = tokenizer.decode(output[0])


    # decoded = tokenizer.batch_decode(output)# Mambas
    # messages.append(dict(
    #     role="assistant",
    #     content=decoded[0].split("<|assistant|>\n")[-1])
    # )
    #output = decoded[0].split("<|assistant|>\n")[-1]
    messages = []

    # # Evaluate ROUGE scores
    #prediction = output.split("<|endoftext|>")[0]


    reference = ref


    #print("Model:", output.split("<|endoftext|>")[0])
    #base_pred = output.split("<|endoftext|>")[0]
    # prediction = prediction.split("<bos>")[0]
    # prediction = prediction.split("<eos>")[0]
    prediction = prediction.split("[/INST]")[1]
    prediction = prediction.split("</s>")[0]
    prediction = prediction.strip()
    predictions.append(prediction)


    print(f"Time: {test_count}/{10}")
    print("Model:", prediction)
    #print(f"ROUGE: {rouge.compute(predictions=[prediction], references=[reference])}")



####################
    rouge_dict = rouge.compute(predictions=[prediction], references=[reference])
    rouge1 += rouge_dict['rouge1']
    rouge2 += rouge_dict['rouge2']
    rougeL += rouge_dict['rougeL']
    rougeLsum += rouge_dict['rougeLsum']

    #print('BLEU score -> {}'.format(sentence_bleu([ref.split()], prediction.split())))
    bleu += sentence_bleu([reference.split()], prediction.split())

    print(f"BLEU: {bleu/(test_count + 1)}")
    print(f"rogue1: {rouge1/(test_count + 1)}")
    print(f"rogue2: {rouge2/(test_count + 1)}")
    print(f"rogueL: {rougeL/(test_count + 1)}")
    print(f"rougeLsum: {rougeLsum/(test_count + 1)}")
##################

    #print(base_pred.split("</s>")[0])

    # print(rouge.compute(predictions=[prediction], references=[reference]))
    # #if test_count == 37365: # Automatically exit
    if test_count == 10:
      break
    test_count += 1


print("Done")

rouge_dict = rouge.compute(predictions=predictions, references=ref_list)
rouge1 = rouge_dict['rouge1']
rouge2 = rouge_dict['rouge2']
rougeL = rouge_dict['rougeL']
rougeLsum = rouge_dict['rougeLsum']

perp = perplexity.compute(predictions=predictions, references=ref_list, model_id="meta-llama/Llama-2-7b-chat-hf")
print(f"Perplexity: {perp}")

print(f"rogue1: {rouge1}")
print(f"rogue2: {rouge2}")
print(f"rogueL: {rougeL}")
print(f"rougeLsum: {rougeLsum}")



Time: 0/10
Model: Sure! Here is the LaTeX code for the given prompt:

\documentclass{article}
\begin{document}
\begin{equation}
\epsilon = \frac{a^2 + 2^2}{c^2 + 4^2}
\end{equation}
\end{document}

Note: This will produce an equation with the given expression on the left-hand side and the right-hand side as the denominator.
BLEU: 0.07978199887317745
rogue1: 0.28169014084507044
rogue2: 0.2608695652173913
rogueL: 0.28169014084507044
rougeLsum: 0.28169014084507044
Time: 1/10
Model: Sure! Here is the LaTeX code for the given prompt:

\documentclass{article}
\begin{document}

\begin{equation}
\epsilon \approx \frac{a^2 + 2^2}{c^2 + 4^2}
\end{equation}

\end{document}
BLEU: 0.09395602612638457
rogue1: 0.3449267030755965
rogue2: 0.30064754856614245
rogueL: 0.3449267030755965
rougeLsum: 0.3449267030755965
Time: 2/10
Model: Sure! Here's the LaTeX code for the given equation:

$$r = \frac{w}{r - t - t - t - t}$$

I hope this helps! Let me know if you have any other questions.
BLEU: 0.06263735075

/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

Time: 3/10
Model: Here is the LaTeX code for the given equation:

\documentclass{article}
\begin{document}
\begin{equation}
r = \frac{w}{r - t - t - t - t} \label{eq:fundamental}
\end{equation}
\end{document}

Note: The equation is typeset using the `equation` environment, which automatically numbers the equation and adds a space between the equation and the label. The label `(\ref{eq:fundamental})` can be used to reference the equation in the text.
BLEU: 0.046978013063192284
rogue1: 0.34632698790143457
rogue2: 0.22106020839159835
rogueL: 0.34632698790143457
rougeLsum: 0.34632698790143457
Time: 4/10
Model: Here is the LaTeX code for the given formula:

$$q = v \times -17 \text{ known as the ideal law}$$
BLEU: 0.03758241045055383
rogue1: 0.41899707419211546
rogue2: 0.25960678740293386
rogueL: 0.40609384838566376
rougeLsum: 0.40609384838566376
Time: 5/10
Model: Here is the LaTeX code for the given formula:

$$q = v \times -17$$

Note: The `-` symbol is used to indicate a negative number 

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 172.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 161.06 MiB is free. Process 8708 has 14.59 GiB memory in use. Of the allocated memory 14.46 GiB is allocated by PyTorch, and 7.32 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)